In [16]:
import os, re 
import numpy as np
import tensorflow as tf
import glob
import os
import numpy as np

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print(np.shape(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
(187088,)
Examples:
 ['[Verse 1]', 'They come from everywhere', 'A longing to be free']


In [17]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.
#     if sentence.count(" ") >= 14 : continue
    if idx > 10: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

[Verse 1]
They come from everywhere
A longing to be free
They come to join us here
From sea to shining sea And they all have a dream
As people always will
To be safe and warm
In that shining city on the hill Some wanna slam the door
Instead of opening the gate
Aw, let's turn this thing around
Before it gets too late [Chorus]


In [18]:
import re
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [19]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if sentence.count(" ") > 10: continue

    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
print(corpus[:10])
print(len(corpus))

['<start> verse <end>', '<start> they come from everywhere <end>', '<start> a longing to be free <end>', '<start> they come to join us here <end>', '<start> from sea to shining sea and they all have a dream <end>', '<start> as people always will <end>', '<start> to be safe and warm <end>', '<start> instead of opening the gate <end>', '<start> aw , let s turn this thing around <end>', '<start> before it gets too late chorus <end>']
151260


In [20]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=20000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  492    3 ...    0    0    0]
 [   2   43   64 ...    0    0    0]
 [   2    9 2967 ...    0    0    0]
 ...
 [   2  558   20 ...    0    0    0]
 [   2  121   36 ...    0    0    0]
 [   2    5   23 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f8f8eb60e90>


In [21]:
print(tensor[:3, :10])
print(tensor[0])

[[   2  492    3    0    0    0    0    0    0    0]
 [   2   43   64   73  706    3    0    0    0    0]
 [   2    9 2967   10   27  263    3    0    0    0]]
[  2 492   3   0   0   0   0   0   0   0   0   0   0   0   0]


In [22]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break
        

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


* 모든 문장이 2로 시작하는 이유를 이걸로 알 수 있음

In [23]:
from sklearn.model_selection import train_test_split

# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(np.shape(tensor))
print(tensor)
print()
print(src_input[0])
print(tgt_input[0])


enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, shuffle = True)

(151260, 15)
[[   2  492    3 ...    0    0    0]
 [   2   43   64 ...    0    0    0]
 [   2    9 2967 ...    0    0    0]
 ...
 [   2  558   20 ...    0    0    0]
 [   2  121   36 ...    0    0    0]
 [   2    5   23 ...    0    0    0]]

[  2 492   3   0   0   0   0   0   0   0   0   0   0   0]
[492   3   0   0   0   0   0   0   0   0   0   0   0   0]


In [24]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (121008, 14)
Target Train: (121008, 14)


In [25]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>


In [26]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

* 우리가 만들 모델에는 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있습니다.

In [27]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [28]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 20001), dtype=float32, numpy=
array([[[-2.51589900e-05, -5.69845106e-05, -1.21771387e-04, ...,
         -1.29070206e-04,  4.07463340e-05, -6.32528245e-05],
        [-3.39247454e-05, -1.94842039e-04, -1.96926339e-04, ...,
         -2.54112791e-04,  1.61474691e-05, -2.90521886e-04],
        [ 1.66235586e-05, -1.50860549e-04, -1.74869274e-05, ...,
         -4.46495396e-04, -1.55285656e-04, -4.58505878e-04],
        ...,
        [ 7.98265566e-04,  5.34281018e-04,  7.45067198e-04, ...,
          1.12606189e-03, -3.96258874e-05, -1.22129830e-04],
        [ 8.68132338e-04,  3.72105598e-04,  9.77526302e-04, ...,
          1.73568737e-03,  9.35570142e-07, -2.84450449e-04],
        [ 9.17812809e-04,  2.28051387e-04,  1.15938811e-03, ...,
          2.29272596e-03,  5.71142009e-05, -4.50045802e-04]],

       [[-2.51589900e-05, -5.69845106e-05, -1.21771387e-04, ...,
         -1.29070206e-04,  4.07463340e-05, -6.32528245e-05],
        [-8.91419113e-05, -5.77505234e-05,  4

* shape=(256, 20, 7001) 7001개의 단어인 이유
    1. 256 : 배치사이즈
    2. 20 : return_sequences=True에서  입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력
    3. 7001 : 7000개의 단어 + \"\<unk>"

In [29]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  5120256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  20501025  
Total params: 39,260,961
Trainable params: 39,260,961
Non-trainable params: 0
_________________________________________________________________


In [31]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=15)

Epoch 1/15
590/590 [==============================] - 247s 415ms/step - loss: 3.5625
Epoch 2/15
590/590 [==============================] - 248s 420ms/step - loss: 2.8245
Epoch 3/15
590/590 [==============================] - 250s 423ms/step - loss: 2.5247
Epoch 4/15
590/590 [==============================] - 250s 424ms/step - loss: 2.2790
Epoch 5/15
590/590 [==============================] - 250s 423ms/step - loss: 2.0748
Epoch 6/15
590/590 [==============================] - 251s 425ms/step - loss: 1.8961
Epoch 7/15
590/590 [==============================] - 250s 424ms/step - loss: 1.7348
Epoch 8/15
590/590 [==============================] - 250s 423ms/step - loss: 1.5872
Epoch 9/15
590/590 [==============================] - 250s 423ms/step - loss: 1.4593
Epoch 10/15
590/590 [==============================] - 250s 423ms/step - loss: 1.3474
Epoch 11/15
590/590 [==============================] - 250s 423ms/step - loss: 1.2531
Epoch 12/15
590/590 [==============================] - 248s 421

In [32]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [44]:
generate_text(model, tokenizer,  init_sentence="<start> i love")

'<start> i love you , <end> '